<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Linear-optimisation-with-scipy" data-toc-modified-id="Linear-optimisation-with-scipy-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Linear optimisation with <code>scipy</code></a></span><ul class="toc-item"><li><span><a href="#Optimisation-in-scipy" data-toc-modified-id="Optimisation-in-scipy-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Optimisation in scipy</a></span></li><li><span><a href="#Constrained-minimization" data-toc-modified-id="Constrained-minimization-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Constrained minimization</a></span></li></ul></li></ul></div>

# Linear optimisation with `scipy`

`scipy` is a python module that has a few of very useful data analysis methods.

There are `scipy` tutorials in [here](https://docs.scipy.org/doc/scipy/reference/tutorial/) lists most of the useful operations of `scipy`. A good overview of examples can be found [here](https://www.guru99.com/scipy-tutorial.html).

Here is a short example – it will be extended in the future!

## Optimisation in scipy 

In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
def rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [ ]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='nelder-mead', options={'xatol': 1e-8, 'disp': True})

In [ ]:
print(res.x)

In [ ]:
def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

In [ ]:
 res = minimize(rosen, x0, method='BFGS', jac=rosen_der, options={'disp': True})

In [ ]:
res.x

In [ ]:
from scipy.optimize import Bounds
bounds = Bounds([0, -0.5], [1.0, 2.0])

In [ ]:
from scipy.optimize import LinearConstraint
linear_constraint = LinearConstraint([[1, 2], [2, 1]], [-np.inf, 1], [1, 1])

In [ ]:
def cons_f(x):
    return [x[0]**2 + x[1], x[0]**2 - x[1]]
def cons_J(x):
    return [[2*x[0], 1], [2*x[0], -1]]
def cons_H(x, v):
    return v[0]*np.array([[2, 0], [0, 0]]) + v[1]*np.array([[2, 0], [0, 0]])
from scipy.optimize import NonlinearConstraint

nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess=cons_H)

In [ ]:
def rosen_hess(x):
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H

In [ ]:
res = minimize(rosen, x0, method='Newton-CG', jac=rosen_der, hess=rosen_hess, options={'xtol': 1e-8, 'disp': True})

In [ ]:
res.x

In [ ]:
x0 = np.array([0.5, 0])
res = minimize(rosen, x0, method='trust-constr', jac=rosen_der, hess=rosen_hess,
               constraints=[linear_constraint, nonlinear_constraint],
               options={'verbose': 1}, bounds=bounds)

In [ ]:
print(res.x)

## Constrained minimization 

In [ ]:
from scipy.optimize import Bounds
bounds = Bounds([0, -0.5], [1.0, 2.0])

In [ ]:
from scipy.optimize import LinearConstraint
linear_constraint = LinearConstraint([[1, 2], [2, 1]], [-np.inf, 1], [1, 1])

In [ ]:
def cons_f(x):
    return [x[0]**2 + x[1], x[0]**2 - x[1]]

def cons_J(x):
    return [[2*x[0], 1], [2*x[0], -1]]

def cons_H(x, v):
    return v[0]*np.array([[2, 0], [0, 0]]) + v[1]*np.array([[2, 0], [0, 0]])

from scipy.optimize import NonlinearConstraint

nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess=cons_H)

In [ ]:
x1 = np.array([2, 1])

res = minimize(rosen, x1, method='trust-constr', jac=rosen_der, hess=rosen_hess, 
                   constraints=[linear_constraint, nonlinear_constraint], 
                   options={'verbose': 1}, bounds=bounds)

In [ ]:
print(res.x)

In [ ]:
print(res)